In [10]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [5]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 8.6 MB/s eta 0:00:00


In [6]:
%pip install --upgrade --quiet  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
OPENAI_API_KEY="api-key"

In [ ]:
OPENAI_API_KEY="api-key"
GROQ_API_KEY="api-key"

In [7]:
pip install langchain-openai langchain-groq gradio pypdf requests python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 12.8 MB/s eta 0:00:00


In [ ]:
GROQ_API_KEY="api-key"

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
import gradio as gr
import requests
import os
import warnings
from dotenv import load_dotenv

# Configuration initiale
warnings.filterwarnings("ignore")
load_dotenv()  # Charge les variables d'environnement depuis .env

# Initialisation sécurisée du modèle Groq
try:
    llm = ChatGroq(
        api_key=os.getenv("GROQ_API_KEY", "api-key"),
        model_name="llama3-8b-8192",
        temperature=0.1,  # Légèrement plus créatif
        max_tokens=4000   # Augmentation des tokens pour de meilleurs résumés
    )
except Exception as e:
    print(f"Erreur d'initialisation Groq: {e}")
    raise

def process_pdf(pdf_url):
    """Télécharge et traite un PDF"""
    try:
        response = requests.get(pdf_url, timeout=10)
        response.raise_for_status()

        temp_path = "temp_pdf.pdf"
        with open(temp_path, "wb") as f:
            f.write(response.content)

        loader = PyPDFLoader(temp_path)
        docs = loader.load()
        os.remove(temp_path)  # Nettoyage du fichier temporaire

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,  # Taille augmentée pour plus de contexte
            chunk_overlap=200,
            separators=["\n\n", "\n", " ", ""]
        )
        return text_splitter.split_documents(docs)
    except Exception as e:
        raise Exception(f"Erreur de traitement PDF: {str(e)}")

def summarize_content(url, content_type):
    """Fonction principale de résumé"""
    try:
        if content_type == "Web Page":
            loader = WebBaseLoader(url)
            docs = loader.load()
            chain = load_summarize_chain(llm, chain_type="stuff")
        elif content_type == "PDF":
            docs = process_pdf(url)
            prompt_template = """Résumez ce contenu en français:
            {text}

            Résumé concis:"""
            prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
            chain = load_summarize_chain(
                llm,
                chain_type="map_reduce",
                map_prompt=prompt,
                combine_prompt=prompt
            )
        else:
            return "Type de contenu non valide"

        if not docs:
            return "Aucun contenu trouvé à résumer"

        return chain.run(docs)

    except Exception as e:
        return f"Erreur: {str(e)}"

# Interface Gradio améliorée
with gr.Blocks(theme=gr.themes.Soft(), title="Résumeur Intelligent") as app:
    gr.Markdown("## 📝 Résumeur de Contenu (Groq)")

    with gr.Row():
        url_input = gr.Textbox(
            label="URL à résumer",
            placeholder="https://exemple.com ou https://exemple.com/document.pdf",
            interactive=True
        )
        content_type = gr.Radio(
            ["Web Page", "PDF"],
            label="Type de contenu",
            value="Web Page"
        )

    submit_btn = gr.Button("Générer le Résumé", variant="primary")

    with gr.Row():
        output = gr.Textbox(
            label="Résumé",
            interactive=False,
            lines=12,
            show_copy_button=True
        )

    # Exemples prédéfinis
    gr.Examples(
        examples=[
            ["https://lilianweng.github.io/posts/2023-06-23-agent/", "Web Page"],
            ["https://arxiv.org/pdf/2303.10130.pdf", "PDF"]
        ],
        inputs=[url_input, content_type],
        label="Exemples de test"
    )

    submit_btn.click(
        fn=summarize_content,
        inputs=[url_input, content_type],
        outputs=output
    )

# Lancement de l'application
try:
    app.launch(share=True)
except Exception as e:
    print(f"Erreur de lancement: {str(e)}")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5c7e770994a1f46072.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


traduire le outout
